# Ansible Galaxy Redfish collections

Version 0.119

## Introduction

The Ansible Galaxy [community.general collection](https://galaxy.ansible.com/community/general) proposes three Redfish modules:
  
  * [Redfish info](https://docs.ansible.com/ansible/latest/collections/community/general/redfish_info_module.html) - Retrieves information like systems and sccounts inventories. The exhaustive list of possible commands is in the `CATEGORY_COMMANDS_ALL` array in the [module sources](https://github.com/ansible-collections/community.general/blob/main/plugins/modules/remote_management/redfish/redfish_info.py). 
  * [Redfish command](https://docs.ansible.com/ansible/latest/collections/community/general/redfish_command_module.html) - Performs `set` operations on log management, user management, and power operations (e.g. on, off, reboot, etc.). The exhaustive list of possible commands is in the `CATEGORY_COMMANDS_ALL` array in the [module sources](https://github.com/ansible-collections/community.general/blob/main/plugins/modules/remote_management/redfish/redfish_command.py) 
  * [Redfish config](https://docs.ansible.com/ansible/latest/collections/community/general/redfish_config_module.html) - Performs configuration operations on BIOS and other subsystems. The exhaustive list of possible commands is in the `CATEGORY_COMMANDS_ALL` array in the [module sources](https://github.com/ansible-collections/community.general/blob/main/plugins/modules/remote_management/redfish/redfish_config.py) 
  
The above Ansible Redfish modules, are based upon the [redfish_utils.py](https://github.com/ansible-collections/community.general/blob/main/plugins/module_utils/redfish_utils.py) python utility module.


Redfish Ansible Playbook examples using the above modules are present in the [DMTF Redfish-Ansible-Playbooks](https://github.com/DMTF/Redfish-Ansible-Playbooks) public GitHub repository.

> **Note**: The Ansible Galaxy Redfish modules can be extended using the [DMTF](https://github.com/DMTF/Redfish-Ansible-Playbooks/blob/master/OEM_EXTENSIONS.md) Oem extensions instructions. This part is not covered in this workshop.


## <a id = "EnvironmentPreparation">Environment preparation</a>

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation (Version: 0.132) ############

# Set Student ID number
stdid={{ STDID }}
Id=$(id --user --name)
NbId=4
InvFile=${NbId}/hosts

# location and ports variables
IloSyBasePort={{ ILOSYBASEPORT }}
let iLO5SimulatorBasePort=$IloSyBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${stdid}

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

# Fake Credentials as we are testing against a BMC simulator
OvSsoToken="FakeOvSsoToken"

# Miscellaneous
WorkshopDir=$PWD
HpePythonRedfishVenv="${NbId}/HpePythonRedfishVenv"
export PYTHONPATH="${WorkshopDir}/${NbId}/library/"
w=$(basename $PWD)

alias ResetSimulators="../create-globalbmc.shc.x &>/dev/null; sleep 1"

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &>/dev/null && 
        echo -e "\n\tGood News: $bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done


# Create the Ansible inventory file
cat > ${InvFile} << __EOF__
[OneViewManagedBmcs]
${iLO5SimulatorIP} ansible_port=${iLO5SimulatorPort}

[OneViewManagedBmcs:vars]
ansible_python_interpreter=${WorkshopDir}/${HpePythonRedfishVenv}/bin/python3
ansible_search_path=${HpePythonRedfishVenv}

# Below is a fake session token as we are testing against an iLO 5 simulator.
# In real life, you should populate this variable with the token obtained
# lab 1 of this workshop.
token="${OvSsoToken}"
__EOF__


## Virtual Python environment creation

In order to completely isolate this notebook environment from other notebooks or student python environments, it is safer to create your dedicated Python virtual environment.

> NOTE: This `Venv` creation can take up to **2 minutes**. Just wait until message `Finished creating Venv` is diplayed



In [ ]:
# Create Virtual Python environment (Venv)
[ -d ${HpePythonRedfishVenv} ] && rm -r ${HpePythonRedfishVenv} &>/dev/null 
python3 -m venv ${HpePythonRedfishVenv}                         &>/dev/null
source ${HpePythonRedfishVenv}/bin/activate                     &>/dev/null
PS1="[PEXP\[\]ECT_PROMPT>"                                      # Avoid Venv long prompt messing up outputs

# Install latest Ansible in the Venv
pip install jmespath                                            &>/dev/null
pip install ansible                                             &>/dev/null

echo -e "\n\n\tFinished creating Venv\n\n"

### <a id = "iLO5SimulatorRestart">Restart iLO 5 simulator</a>

If you need or desire to restart your iLO 5 simulator to restart this workshop from scratch or for other reasons, run the following cell at any time.

In [ ]:
# iLO 5 Simulator restart
ResetSimulators 

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

## Get and set Redfish properties using Ansible Galaxy collections

This third way of using Ansible for Redfish tasks uses the `redfish_info` and `redfish_command` collections to get and set the Indicator LED of a Synergy compute node.

Although the Redfish Ansible collections presented here are very handy in RedHat Tower and AWX environments, they don't offer (as of the writing of this notebook) the possibility to use session based authentication (i.e. OneView Token). 

Despite this limitation, it is importand to know the existence of those collection and to know that they can be extended using the  [DMTF instructions](https://github.com/DMTF/Redfish-Ansible-Playbooks/blob/master/OEM_EXTENSIONS.md).


### Indicator LED 

The next cell launches an Ansible Playbook which has the same three sections as the previous plabooks:
  * Get status of indicator LEDs 
  * Modify indicator LED(s) status
  * Verify that the action was successul.


TBD

In [ ]:
# Set Indicator LED
ansible-playbook -i ${InvFile} ${NbId}/SetIndicatorLEDUsingGalaxy.yml

# ****  NEED To Decide wether the following is relevant in this workshop ****

**Pro**: TBD
**Cons**: TBD


## Redfish info example: category Manager

In [ ]:
# Get Virtual Media information
ansible-playbook -i ${InvFile} ${NbId}/GetVirtualMediaInfo.yml

## Insert Virtual Media

In [ ]:
ansible-playbook -i ${InvFile} ${NbId}/InsertVirtualMedia.yml

# ****  END of NEED To Decide wether the following is relevant in this workshop ****

## Summary

In this workshop, you TBD Go to the [Conclusion Notebook](5-Conclusion.ipynb) if you are finished.